In [7]:
import numpy as np
import matplotlib.pyplot as plt
from simulate import *



import matplotlib 
matplotlib.rc('xtick', labelsize=27) 
matplotlib.rc('ytick', labelsize=27)

In [8]:
def sim_data(inclination_earth, P_jup, roman_err, roman_duration, gaia_obs):

	'''
	inc_earth = Earth inclination in degrees (Jupiter assumed 1.31 degrees greater than this value)
	period_jup = Jupiter period in days 
	roman_err = roman error in arcseconds, if None assumed no Roman observations
	'''




	##################
	##################
	##################
	##################
	#begin simulate data
	##################
	##################
	##################
	##################
	##################

	T_subtract = 2454000
	# orbital parameters from https://www.princeton.edu/~willman/planetary_systems/Sol/
	# BJD determined by converting values above using https://ssd.jpl.nasa.gov/tc.cgi#top

	P_earth = 300
	e_earth = 0.0167
	Tper_earth= 100
	omega_earth = np.radians(102.9)
	Omega_earth = np.radians(0.0)
	inclination_earth = np.radians(inc_earth)
	m_earth = 1*3.00273e-6 #units m_sun



	P_jup = period_jup	
	e_jup = 0.0484
	Tper_jup = 500
	omega_jup = np.radians(274.3) - 2*np.pi
	Omega_jup = np.radians(100.4)
	inclination_jup = np.radians(1.31) + inclination_earth
	m_jup = 317.83*3.00273e-6 #units m_sun


	m_sun = 333030 #earth masses


	#add gaia observing times
	times_observed_astrometry_gaia = []
	t_0 = int(Tper_earth)
	for ii in range(t_0, t_0+3600):
		if ii % (3600/gaia_obs) == 0:
			times_observed_astrometry_gaia.append(ii)

	
			
	#add THE observing times
	times_observed_rv = []
	t_0 = int(Tper_earth)
	add_data = True
	for ii in range(t_0, t_0+3600):
		
		if ii % 180 == 0:
			if add_data:
				add_data = False
			else:
				add_data = True
		   
		if add_data:
			times_observed_rv.append(ii)
			

	orbit_params_earth = [P_earth, e_earth, Tper_earth, omega_earth, Omega_earth, inclination_earth, m_earth]
	orbit_params_jup = [P_jup, e_jup, Tper_jup, omega_jup, Omega_jup, inclination_jup, m_jup]

	n_planets = 2
	orbit_params = [orbit_params_earth, orbit_params_jup]


	sigma_rv = 0.3

	sigma_ra_gaia = 6e-5
	sigma_dec_gaia = 6e-5
	parallax = 0.1 #as



	times, rv_results, ra_results, dec_results = simulate_data(
		n_planets, 
		sigma_rv, 
		sigma_ra_gaia,
		sigma_dec_gaia,
		parallax,
		orbit_params,
		times_observed_rv = times_observed_rv,
		times_observed_astrometry = times_observed_astrometry_gaia
		)


	[[times_rv, times_observed_rv, times_astrometry, times_observed_astrometry],
	[rv_orbit, rv_orbit_sum, rv_sim, rv_sim_sum],
	[ra_orbit, ra_orbit_sum, ra_sim, ra_sim_sum],
	[dec_orbit, dec_orbit_sum, dec_sim, dec_sim_sum]]  = times, rv_results, ra_results, dec_results

	ra_gaia_err = np.full(np.shape(ra_sim_sum), sigma_ra_gaia)
	dec_gaia_err = np.full(np.shape(dec_sim_sum), sigma_dec_gaia)


	#add roman observing times if roman_err not None
	if roman_err is not None:
		t_1 =  times_observed_astrometry_gaia[-1]+1800
		times_observed_astrometry_roman = []
		for ii in range(t_1, t_1+(roman_duration*365)):
			if ii % 90 == 0:
				times_observed_astrometry_roman.append(ii)	


		sigma_ra_roman = roman_err
		sigma_dec_roman = roman_err



		times, rv_results, ra_results, dec_results = simulate_data(
			n_planets, 
			sigma_rv, 
			sigma_ra_roman,
			sigma_dec_roman,
			parallax,
			orbit_params,
			times_observed_rv = times_observed_rv,
			times_observed_astrometry = times_observed_astrometry_roman
			)

		times_astrometry = np.append(times_astrometry, times[2], axis=0)

		times_observed_astrometry = np.append(times_observed_astrometry, times[3], axis=0)

		ra_orbit = np.append(ra_orbit, ra_results[0], axis=0)
		ra_orbit_sum = np.append(ra_orbit_sum, ra_results[1], axis=0)
		ra_sim = np.append(ra_sim, ra_results[2], axis=0)
		ra_sim_sum = np.append(ra_sim_sum, ra_results[3], axis=0)

		dec_orbit = np.append(dec_orbit, dec_results[0], axis=0)
		dec_orbit_sum = np.append(dec_orbit_sum, dec_results[1], axis=0)
		dec_sim = np.append(dec_sim, dec_results[2], axis=0)
		dec_sim_sum = np.append(dec_sim_sum, dec_results[3], axis=0)

		ra_roman_err = np.full(np.shape(ra_results[3]), sigma_ra_roman)
		dec_roman_err = np.full(np.shape(dec_results[3]), sigma_dec_roman)


	
	##################
	##################
	##################
	##################
	#end simulate data
	##################
	##################
	##################
	##################
	##################



	##################
	##################
	##################
	##################
	#begin model data
	##################
	##################
	##################
	##################
	##################

	################
	################
	#rename variables in more consistent way for modeling
	x_rv = np.array(times_observed_rv)
	y_rv = rv_sim_sum
	y_rv_err = np.full(np.shape(y_rv), sigma_rv)

	x_astrometry = np.array(times_observed_astrometry)
	ra_data = ra_sim_sum
	dec_data = dec_sim_sum


	if roman_err is not None:
		ra_err = np.concatenate((ra_gaia_err, ra_roman_err))
		dec_err = np.concatenate((dec_gaia_err, dec_roman_err))

	else:
		ra_err = ra_gaia_err
		dec_err = dec_gaia_err



	# make a fine grid that spans the observation window for plotting purposes
	t_astrometry = np.linspace(x_astrometry.min() - 5, x_astrometry.max() + 5, 1000)
	t_rv = np.linspace(x_rv.min() - 5, x_rv.max() + 5, 1000)

	# for predicted orbits
	t_fine = np.linspace(x_astrometry.min() - 500, x_astrometry.max() + 500, num=1000)
    
	return x_rv, t_rv, y_rv, y_rv_err, x_astrometry, t_astrometry, t_fine, ra_data, dec_data, ra_err, dec_err



In [9]:
x_rv, t_rv, y_rv, y_rv_err, x_astrometry, t_astrometry, t_fine, ra_data, dec_data, ra_err, dec_err = sim_data(45., 4327, 5e-6, 10, 200)

In [10]:
files = [
'period4327_inc45_gaia60_roman5_5.pkl',
'period4327_inc45_gaia60_roman5_10.pkl',
'period4327_inc45_gaia60_roman10_5.pkl',
'period4327_inc45_gaia60_roman10_10.pkl',
'period4327_inc45_gaia60_roman20_5.pkl',
'period4327_inc45_gaia60_roman20_10.pkl',
'period4327_inc45_gaia60_romanNA.pkl'
]

In [ ]:
P_earth = 300
e_earth = 0.0167
Tper_earth= 100
omega_earth = np.radians(102.9)
Omega_earth = np.radians(0.0)
inclination_earth = np.radians(inc_earth)
m_earth = 1*3.00273e-6 #units m_sun



P_jup = period_jup
e_jup = 0.0484
Tper_jup = 500
omega_jup = np.radians(274.3) - 2*np.pi
Omega_jup = np.radians(100.4)
inclination_jup = np.radians(1.31) + inclination_earth
m_jup = 317.83*3.00273e-6 #units m_sun


m_sun = 333030 #earth masses

In [11]:
import pickle

with open('./traces/Sep22/period4327_inc45_gaia60_roman5_10.pkl', 'rb') as buff:
    data = pickle.load(buff)  

model, trace = data['model'], data['trace']





In [12]:
from model import a_from_Kepler3

a_true_earth = a_from_Kepler3(P_earth, 1.0+m_earth)
a_true_jup = a_from_Kepler3(P_jup, 1.0+m_jup)

truth_both = [P_earth, P_jup, e_earth, e_jup, omega_earth, omega_jup, Omega_earth, Omega_jup, 
              inclination_earth, inclination_jup, 0.1, m_earth*m_sun, m_jup*m_sun]


import corner

_ = corner.corner(
    trace, var_names=["P", "ecc", "omega", "Omega", "incl", 
                      "plx", "m_planet"], quantiles=[0.16, 0.5, 0.84],
                       show_titles=True, title_kwargs={"fontsize": 13}, 
                       truths = truth_both, truth_color = "#00257c",
                       label_kwargs={"fontsize": 13}
)

plt.savefig('./final_figures/corner.pdf')

NameError: name 'P_earth' is not defined

In [ ]:
matplotlib.rc('xtick', labelsize=36) 
matplotlib.rc('ytick', labelsize=36)

ekw = dict(fmt=".k", lw=0.5)

fig, ax = plt.subplots(nrows=2, sharex=True, figsize = [18,13], gridspec_kw={'height_ratios': [2, 2]})

ax[0].set_ylabel(r"$\Delta \alpha \cos \delta$ ['']", fontsize = 36)
ax[1].set_ylabel(r"$\Delta \delta$ ['']", fontsize = 36)

ax[1].set_xlabel(r"time [days]", fontsize = 36)

tot_ra_err = ra_err 
tot_dec_err = dec_err

ax[0].errorbar(x_astrometry, ra_data, yerr=tot_ra_err, **ekw, 
               color = '#00257c', markersize=5)
q = np.percentile(trace.posterior["ra_model_fine"].values, [16, 84], axis=(0, 1))
ax[0].fill_between(t_fine, q[0], q[1], color="k", alpha=0.5, lw=.1)




ax[1].errorbar(x_astrometry, dec_data, yerr=tot_dec_err, **ekw, 
               color = '#00257c', markersize=5)
q = np.percentile(trace.posterior["dec_model_fine"].values, [16, 84], axis=(0, 1))
ax[1].fill_between(t_fine, q[0], q[1], color="k", alpha=0.5, lw=.1)



ax[0].set_xlim(t_fine[0], t_fine[-1])
_ = ax[0].set_title("MCMC Posterior to Astrometry Data", fontsize = 45)



ax[0].set_ylim(-0.00065, 0.0007)  
ax[1].set_ylim(-0.00065, 0.00065)

ax[0].axvline(4700, ymin = 0, ymax = 1, color = 'k', ls = '--')
ax[0].text(4600, 0.0006, "Gaia", fontsize = 27, ha='right')
ax[0].text(4800, 0.0006, "Roman", fontsize = 27, ha='left')
ax[0].arrow(4600, 0.00055, -600, 0, width = 0.00001, 
          head_width= 0.00005, head_length = 100, color ='k')
ax[0].arrow(4800, 0.00055, 900, 0, width = 0.00001, 
          head_width= 0.00005, head_length = 100, color ='k')


ax[1].axvline(4700, ymin = 0, ymax = 1, color = 'k', ls = '--')
ax[1].text(4600, 0.0005, "Gaia", fontsize = 27, ha='right')
ax[1].text(4800, 0.0005, "Roman", fontsize = 27, ha='left')
ax[1].arrow(4600, 0.00045, -600, 0, width = 0.00001, 
          head_width= 0.00005, head_length = 100, color ='k')
ax[1].arrow(4800, 0.00045, 900, 0, width = 0.00001, 
          head_width= 0.00005, head_length = 100, color ='k')


plt.tight_layout()
plt.savefig('final_figures/astrometry_MCMC.pdf')